In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, os, django
import math


from sklearn.manifold import TSNE
from sklearn.svm import SVC, OneClassSVM
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, recall_score

from sklearn.decomposition import NMF
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from scipy.sparse import hstack

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
from sklearn.manifold import TSNE
from scoping.models import *
from scoping.views import *

import learning_utils
import random


double_figure = (7.2,7.2/8*5)
plt.rc('font',size=7)
plt.rc('axes',titlesize=7)
plt.rc('axes',labelsize=7)
plt.rc('xtick',labelsize=7)
plt.rc('ytick',labelsize=7)
plt.rc('legend',fontsize=7)
plt.rc('figure',titlesize=7)

In [7]:
q = Query.objects.get(pk=7638)

In [11]:
os.listdir("doctables")

['db1_docs.csv',
 'predictions_2020-01-21 15:38.csv',
 'doc_numbers.csv',
 'predictions_2020-01-29 15:56.csv',
 'predictions_2020-01-21 15:36.csv',
 'db1_docs_long.csv',
 'predictions_2020-02-18 12:50.csv',
 'db1_docs_very_long.csv']

In [15]:
dodf = (pd.DataFrame.from_dict(DocOwnership.objects.filter(query=q).values('doc__id','relevant'))
        .rename(columns={"doc__id":"id","relevant":"rating"})
       )
print(dodf.shape)
dodf.head()

(10280, 2)


,id,rating
0,2472514,1
1,2461354,2
2,2428397,2
3,2390102,1
4,2351316,2


In [37]:
for f in sorted(os.listdir("doctables")):
    if "predictions" in f:
        print(f)
        fdf = (pd.read_csv(f"doctables/{f}")
               .query("prediction.notnull()", engine="python")
               .merge(dodf, how="inner")
              )
        print(fdf.shape)
        mispredicted = fdf[
            ((fdf['prediction']>0.5) & (fdf['rating']!=1)) |
            ((fdf['prediction']<0.5) & (fdf['rating']==1))
        ]
        print(mispredicted.shape)
    

predictions_2020-01-21 15:36.csv
(697, 9)
(252, 9)
predictions_2020-01-21 15:38.csv
(697, 9)
(252, 9)
predictions_2020-01-29 15:56.csv
(400, 9)
(160, 9)
predictions_2020-02-18 12:50.csv
(8160, 9)
(2924, 9)


In [35]:
mispredicted.head()

,id,content,title,wosarticle__de,wosarticle__wc,relevant,seen,prediction,rating
